# Chess App

In [ ]:
#| default_exp chess_app

In [ ]:
! uv pip install chess stockfish

## Todos
- Use websockets to alert player when another one has joined (opened a second browser and connected)
- Use websockets to alert player when another one has left (closed the browser)
- Use websockets to alert player when another one has made a move
- Add evaluation bar via stockfish chess engine to show who is winning

In [41]:
import chess
bboard = chess.BaseBoard()

In [43]:
type(bboard.piece_at(chess.parse_square('a4')))
# chess.svg.piece(())

NoneType

In [10]:
#| export
from fasthtml import *
from fastcore.utils import *
from starlette.endpoints import WebSocketEndpoint
from starlette.routing import WebSocketRoute

import chess
import chess.svg

bboard = chess.BaseBoard()

css = Style(
    '''\
    #chess-board { display: grid; grid-template-columns: repeat(8, 64px); grid-template-rows: repeat(8, 64px);gap: 1px; }
    .board-cell { width: 64px; height: 64px; border: 1px solid black; }
    .black { background-color: grey; }
    .white { background-color: white; }
    .active { background-color: green; }
    '''
)
# Flexbox CSS (http://flexboxgrid.com/)
gridlink = Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/flexboxgrid/6.3.1/flexboxgrid.min.css", type="text/css")
htmx_ws = Script(src="https://unpkg.com/htmx-ext-ws@2.0.0/ws.js")

app = FastHTML(hdrs=(gridlink, css, htmx_ws,))
rt = app.route

In [ ]:
#| export
player_queue = []
class WS(WebSocketEndpoint):
    encoding = 'text'

    # async def on_connect(self, websocket):
    #     await websocket.accept()

    # async def on_connect(self, websocket):
    #     # global player_queue
    #     # player_queue.append(websocket)
    #     await websocket.accept()
        # await websocket.send_text("hi")
        # if len(player_queue) == 2:
        #     await player_queue[0].send_text("Opponent joined!")
        #     await player_queue[1].send_text("You joined!")

    async def on_receive(self, websocket, data):
        await websocket.send_text("hi")

    # async def on_disconnect(self, websocket, close_code):
    #     pass
        # global player_queue
        # player_queue.remove(websocket)

app.routes.append(WebSocketRoute('/chess', WS))

In [11]:
#| export
def Board():
    def create_board():
        rows = '87654321'
        cols = 'abcdefgh'
        board = []
        for row in rows:
            board_row = []
            for col in cols:
                pos = f"{col}{row}"
                cell_color = "black" if (rows.index(row) + cols.index(col)) % 2 == 0 else "white"
                piece = bboard.piece_at(chess.parse_square(pos))
                if piece:
                    piece = NotStr(chess.svg.piece(piece))
                    board_row.append(
                        Div(
                            piece, id=f"{col}{row}", cls=f"board-cell {cell_color}", hx_post="/select", hx_vars=f"from={pos}&to=e2",
                            hx_swap='innerHTML', hx_target='#gol', hx_trigger='click'
                        )
                    )
                else:
                    board_row.append(Div(id=f"{col}{row}", cls=f"board-cell {cell_color}"))
            board.append(Div(*board_row, cls="board-row"))
        return Div(*board, id="chess-board")

    html_board = create_board()
    return html_board

In [ ]:
#| export
def Home():
    return Div(
        Div('Hello, still waiting on an opponent!', id='user-message'),
        Board(),
        hx_ext="ws", ws_connect="/chess"
    )

In [ ]:
#| export
@rt("/")
def get():
    return Home()# Div(NotStr(chess.svg.piece(chess.Piece.from_symbol("R"))))

In [62]:
#| eval: false
#| hide
from nbdev.export import nb_export
nb_export('chess_app.ipynb', '.')